In [1]:
import numpy as np
import pandas as pd
import os
os.chdir('../')
from gravity import Gravity, Production, Attraction, Doubly, BaseGravity
import statsmodels.formula.api as smf
from statsmodels.api import families
import matplotlib.pyplot as plt
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:

n = 3000
o = np.tile(np.arange(n),n)
d = np.repeat(np.arange(n),n)
loc_size = np.random.randint(25000,500000, n)
o_vars = np.tile(loc_size, n)
d_vars = np.repeat(loc_size, n)
dij = np.random.exponential(2500, n**2)
f = o_vars**.3*d_vars**.4*np.exp(dij*-.00005)
o = np.reshape(o, (-1,1))
d = np.reshape(d, (-1,1))
o_vars = np.reshape(o_vars, (-1,1))
d_vars = np.reshape(d_vars, (-1,1))
dij = np.reshape(dij, (-1,1))
f = np.reshape(f, (-1,1))
f = f.astype(np.int64)

In [3]:
%time Gravity(f, o_vars, d_vars, dij, 'exp')

CPU times: user 3.25 s, sys: 1.15 s, total: 4.4 s
Wall time: 4.26 s


In [4]:
%time Production(f, o, d_vars, dij, 'exp')

CPU times: user 32.4 s, sys: 5.34 s, total: 37.8 s
Wall time: 24.3 s


In [5]:
%time Attraction(f, d, o_vars, dij, 'exp')

CPU times: user 28.7 s, sys: 5.05 s, total: 33.7 s
Wall time: 20 s


In [6]:
%time Doubly(f, o, d, dij, 'exp')

CPU times: user 2min 1s, sys: 7.06 s, total: 2min 8s
Wall time: 40.4 s


In [3]:
from glm import GLM
from iwls import iwls
import line_profiler
import IPython
ip = IPython.get_ipython()
ip.define_magic('lprun', line_profiler.magic_lprun)
instance = Production(f, o, d_vars, dij, 'exp')


In [9]:
%lprun -f BaseGravity.__init__ instance.__init__(f, o, d_vars, dij, 'exp')

correct sparse


In [8]:
glm_inst = GLM(instance.y, instance.X, family=families.Poisson())
%lprun -f GLM.__init__ glm_inst.__init__(instance.y, instance.X, family=families.Poisson())

In [7]:
%lprun -f iwls iwls(instance.y, instance.X, family=families.Poisson(), offset=None, y_fix=None)